### Network growth

1. Search "History" in section headings.
2. Find a list of years.
3. Use the first one as when the birth year of the idea.
4. If no "History" section, then look backwards in the graph to a page that does, and use that.

#### Visualization
* Use ~~[bokeh](https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html)~~ ~~[webweb](https://webwebpage.github.io/)~~ [d3](https://observablehq.com/@d3/force-directed-graph).
* Use slider to show growth in network

### Label with years

In [ ]:
import networkx as nx

path_saved = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/prelim data/'
# natural & physical sciences
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology']
topics += ['chemistry', 'biophysics', 'energy', 'optics', 
           'earth science', 'geology', 'meteorology']
# philosophy
# topics += []
topics += ['philosophy of language', 'philosophy of law', 
           'philosophy of mind', 'philosophy of science']
# social sciences
topics += ['economics', 'accounting', 'education', 'linguistics', 'law', 'psychology', 'sociology']
# technology & applied sciences
topics += ['electronics', 'software engineering', 'robotics']
topics += ['physics']

graphs = {}
for topic in topics:
    graphs[topic] = nx.read_gexf(path_saved + topic + '_d2.gexf')
graphs

In [ ]:
import wiki

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [ ]:
_graphs = {'robotics': graphs['robotics']}
for topic, graph in graphs.items():
    print('Topic: ' + topic)
    for node in graph.nodes:
        dump.load_page(node)
        graph.nodes[node]['year'] = dump.years[0] if len(dump.years)>0 else None

In [ ]:
graphs['robotics'].nodes['Robotics']

#### Fill in nodes without years

In [ ]:
def fill_empty_nodes(graph, full_parents=True):
    """
    Returns
    -------
    bool
        whether at least 1 empty node was filled
    """
    empty_nodes = [n for n in graph.nodes if not graph.nodes[n]['year']]
    for node in empty_nodes:
        years = [graph.nodes[p]['year'] for p in graph.predecessors(node)]
        if not years:
            continue
        if full_parents:
            if [] not in years:
                graph.nodes[node]['year'] = max(years)
                return True
        else:
            years_filtered = [y for y in years if y]
            if years_filtered:
                graph.nodes[node]['year'] = max(years_filtered)
                return True
    return False

In [ ]:
_graphs = {'biochemistry': graphs['biochemistry']}
for topic, graph in _graphs.items():
    nodes = graph.nodes
    print('Filling in nodes with full parents...')
    nodes_filled = True
    while nodes_filled:
        nodes_filled = fill_empty_nodes(graph, full_parents=True)
    print('Filling in nodes without full parents...')
    nodes_filled = True
    while nodes_filled:
        nodes_filled = fill_empty_nodes(graph, full_parents=False)

#### Save graphs

In [ ]:
for topic, graph in _graphs.items():
    path_save = path_base + 'graphs/dated_' + 'filled_' + topic + '.gexf'
    nx.write_gexf(graphs[topic], path_save)

#### Load graphs

In [ ]:
topics = ['biochemistry']
path_base = '/Users/harangju/Developer/data/wiki/graphs/'

In [2]:
import networkx as nx

graphs = {}
for topic in topics:
    graphs[topic] = nx.read_gexf(path_base + 'dated_filled_' + topic + '.gexf')
graphs

{'biochemistry': <networkx.classes.digraph.DiGraph at 0x106a4cf90>}

### Visualize growth

* gephi [tutorial](https://seinecle.github.io/gephi-tutorials/generated-html/converting-a-network-with-dates-into-dynamic.html)

### Clique community growth

In [ ]:
import networkx as nx

In [ ]:
# cliques = nx.enumerate_all_cliques(nx.Graph(graphs['biochemistry']))
cliques = nx.find_cliques(nx.Graph(graphs['biochemistry']))
list(cliques)[100:110]

### Persistent homology